# Add cross-references to other databases

In [1]:
from cobramod.core.crossreferences import add_crossreferences

from cobra.io import write_sbml_model, read_sbml_model

import cobra
import pandas as pd

In [2]:
model = read_sbml_model("../models/RcH10_v3.xml")

In [3]:
add_crossreferences(object=model,directory="/Users/idunmariaburgos/Ruminiclostridium_cellulolyticum_model/Ruminiclostridium-cellullolyticum-model-final/notebooks/")


100%|██████████| 2594/2594 [02:39<00:00, 16.22it/s] 


In [4]:
write_sbml_model(model,"../models/RcH10_v4.xml")

### Annotation of genes

In [2]:
model = read_sbml_model("../models/RcH10_v4.xml")

https://identifiers.org/inchi/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/inchikey/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/inchi/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/inchikey/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/inchi/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/inchikey/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/inchi/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/inchikey/ does 

In [20]:
#uniprot = pd.read_csv("old_notebooks/uniprotkb_taxonomy_id_1521_2024_10_21.tsv",sep="\t")
uniprot = pd.read_csv("../input/uniprot-proteome_UP000001349.tab",sep="\t")
uniprot = uniprot[~uniprot["Cross-reference (RefSeq)"].isna()]
uniprot

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
0,B8I4G1,LEUD_RUMCH,3-isopropylmalate dehydratase small subunit (E...,leuD Ccel_0127,WP_012634581.1;
1,B8I8F2,UVRC_RUMCH,UvrABC system protein C (Protein UvrC) (Excinu...,uvrC Ccel_0807,WP_015924347.1;
2,B8I567,UPP_RUMCH,Uracil phosphoribosyltransferase (EC 2.4.2.9) ...,upp Ccel_0260,WP_012634712.1;
3,B8I364,PDXT_RUMCH,Pyridoxal 5'-phosphate synthase subunit PdxT (...,pdxT Ccel_1859,WP_015925312.1;
4,B8I176,RL21_RUMCH,50S ribosomal protein L21,rplU Ccel_1320,WP_015924822.1;
...,...,...,...,...,...
3281,B8I5G6,B8I5G6_RUMCH,Uncharacterized protein,Ccel_2369,WP_015925793.1;
3282,B8I8G0,B8I8G0_RUMCH,Lytic transglycosylase catalytic,Ccel_0815,WP_015924355.1;
3283,B8I473,B8I473_RUMCH,5-formyltetrahydrofolate cyclo-ligase (EC 6.3....,Ccel_2164,WP_015925601.1;
3285,B8I126,B8I126_RUMCH,Uncharacterized protein,Ccel_3046 Ccel_3293,WP_015926397.1;


In [21]:
for gene in model.genes:
    id_ = gene.id
    annotation = gene.annotation

    annotation_id = uniprot[uniprot["Cross-reference (RefSeq)"].str.contains(id_[:-2])]
    if annotation_id.size==0:
        continue
    
    refseq_id = annotation_id["Cross-reference (RefSeq)"].values[0][:-3] if len(annotation_id["Cross-reference (RefSeq)"].values)>0 else False
    uniprot_id = annotation_id["Entry"].values[0] if len(annotation_id["Entry"].values)>0 else False
    kegg_id = annotation_id["Gene names"].values[0].split(" ")[-1] if len(annotation_id["Gene names"].values)>0 else False
    #print((refseq_id,uniprot_id,kegg_id))

    if refseq_id: annotation["refseq"]=refseq_id #+".1"
    if uniprot_id: annotation["uniprot"]=uniprot_id
    if kegg_id: annotation["kegg.genes"]="cce:"+kegg_id

    gene.annotation=annotation

In [22]:
write_sbml_model(model,"../models/RcH10_v5.xml")

### Fixing URIs in metadata

In [5]:
! source activate rcell_rec;python "../scripts/fix_annotation.py"

loading models...
fixing metabolites...
checking URIs for metabolites...
 Having problems with following URIs:
	 Timeout for https://identifiers.org/inchi:InChI=1S/C20H23N7O7/c21-20-25-16-15(18(32)26-20)23-11(7-22-16)8-27(9-28)12-3-1-10(2-4-12)17(31)24-13(19(33)34)5-6-14(29)30/h1-4,9,11,13,23H,5-8H2,(H,24,31)(H,29,30)(H,33,34)(H4,21,22,25,26,32)/p-2/t11-,13+/m1/s1. Retrying...
	 Failed to connect to https://identifiers.org/inchi:InChI=1S/C20H23N7O7/c21-20-25-16-15(18(32)26-20)23-11(7-22-16)8-27(9-28)12-3-1-10(2-4-12)17(31)24-13(19(33)34)5-6-14(29)30/h1-4,9,11,13,23H,5-8H2,(H,24,31)(H,29,30)(H,33,34)(H4,21,22,25,26,32)/p-2/t11-,13+/m1/s1 after 5 retries.
	 Removed problematic URIs for M_10fthf_c
	 Timeout for https://identifiers.org/CHEBI:28972. Retrying...
	 Timeout for https://identifiers.org/CHEBI:352. Retrying...
	 Timeout for https://identifiers.org/CHEBI:44863. Retrying...
	 Timeout for https://identifiers.org/CHEBI:18705. Retrying...
	 Timeout for https://identifiers.org/inchi:In

In [13]:
model_cobra = cobra.io.read_sbml_model("../models/RcH10_final.xml")

cobra.io.write_sbml_model(model_cobra,"../models/iIB728.xml")